In [1]:
import numpy as np
import functions as fn
from circuit_obj import Circuit
from tqdm import tqdm
from itertools import product

In [2]:
# Define your parameter arrays
N = 16
T = 1000
masks_dict = fn.load_mask_memory(N, 2)

In [3]:
def initial_state(N, n):
    """
    Generate a random state vector on N qubits with spin up at site n
    """
    up = np.array([0, 1])
    if n == 0:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(up, rnd_part)
        state /= np.linalg.norm(state)
        return state
    elif n == N - 1:
        rnd_part = np.random.rand(2**(N - 1)) + 1j * np.random.rand(2**(N - 1))
        state = np.kron(rnd_part, up)
        state /= np.linalg.norm(state)
        return state
    part1 = np.random.rand(2**(n)) + 1j * np.random.rand(2**(n))
    part2 = np.random.rand(2**(N - n - 1)) + 1j * np.random.rand(2**(N - n - 1))
    state = np.kron(part1, np.kron(up, part2))
    state /= np.linalg.norm(state)
    return state

# sanity check :
# [np.round(fn.get_magnetization(initial_state(N, idx), N)[idx]) for idx in range(N)]

st = initial_state(N, 0)

In [12]:
import json

with open('../random_archive/angles_N16.json', 'r', encoding='utf-8') as f:
    loaded_dict = json.load(f)
    
circuits = []
circuit_realizations = 25

for circuit_realization in range(circuit_realizations):
    gates = []
    for n in range(N):
        params = loaded_dict[circuit_realization][n]
        Jz = params['Jz']; Jx = params['Jx']
        θ1 = params['θ1']; θ2 = params['θ2']
        θ3 = params['θ3']; θ4 = params['θ4']
        gates.append(fn.gate_xyz_disordered(θ1, θ2, Jx/2, -Jx/2, Jz/2, θ3, θ4)) # h1, h2, Jx, Jy, Jz, h3, h4
    
    gates = np.array(gates)
    assert len(gates) == N, f"Expected {N} gates, got {gates.shape}"

    order = fn.gen_gates_order(N)    
    circuit = Circuit(N=N, gates=gates, order=order)
    circuits.append(circuit)
    
##############################################################################################

def compute_circuit(idx, circuit_real):
    circuit = circuits[circuit_real]
    state = initial_state(N, idx) # initial_state_test(theta)
    return circuit.run(masks_dict, state, T)

In [15]:
circuits[0].run(masks_dict, initial_state(N, 0), 2)[1].shape

100%|██████████| 2/2 [00:00<00:00, 18.53it/s]


(65536,)

In [13]:
if globals().get('result') is None or globals(
    ).get('result').shape != (N, circuit_realizations, T + 1, N):
    result = np.zeros((N, circuit_realizations, T + 1, N), dtype=np.float64)

if globals().get('final_sv') is None or globals(
    ).get('final_sv').shape != (N, circuit_realizations, T + 1, N):
    final_sv = np.zeros((N, circuit_realizations, 2**N), dtype=np.complex128)

for idx, circuit_real in (product(
        range(N), 
        range(circuit_realizations), #
    )): #, total=1 * circuit_realizations
    print(f"Computing circuit {circuit_real} for initial state {idx}")
    result[idx, circuit_real], final_sv[idx, circuit] = compute_circuit(idx, circuit_real);

Computing circuit 0 for initial state 0


100%|██████████| 1000/1000 [00:57<00:00, 17.44it/s]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# # np.save('result_N16.npy', result)
# result = np.load('result_N16.npy')

In [ ]:
with open("nathan.txt", "r") as f:
    lines = f.readlines()                    # read all lines as strings
data =[float(i) for i in lines[0].split(', ')]  # drop blank lines
print(data)

In [ ]:
np.array(data).shape

In [ ]:
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

%matplotlib osx
new_res = np.zeros((T + 1), dtype=np.float64)

pts = np.linspace(0, T, 10000)
plt.plot(pts, pts**(-.22)/2, color='k', linestyle='--', label=r'$t^{-.22}$')
plt.axhline(1/14, color='k', linestyle=':')
plt.axhline(1/15, color='k', linestyle=':')
plt.axhline(1/16, color='k', linestyle='--')

# plt.plot(np.mean([result[i, :, :, i] for i in range(10)], axis=(0,1)), label='Mean Magnetization')
plt.plot(gaussian_filter(result[0, 0, :, 0], 10), label='Mean Magnetization')
plt.plot(data, label='Nathan Magnetization')
plt.xlabel('Time')
plt.ylabel('Magnetization')
plt.xscale('symlog')
plt.legend()
# plt.yscale('log')


In [ ]:
data[-10:], 1/(13)

In [ ]:
np.mean([result[i, -1, 0, i] for i in range(N)], axis=0)

In [ ]:
[result[i, -1, 0, i] for i in range(N)]

### Check that gates match

In [ ]:
import functions as fn
import numpy as np
sx, sy, sz, id_ = fn.X, fn.Y, fn.Z, fn.I

def print_paulis(gave_evo):
    PAULI = {'Z': sz, 'Y': sy, '1': id_, 'X': sx, }
    for i in PAULI:
        pi = PAULI[i]
        for j in PAULI:
            pj = PAULI[j]; overlap = np.trace(gave_evo @ np.kron(pi, pj))
            if np.abs(overlap) > 1e-10:
                print(i,j,f' -> {overlap.real:.9f}', sep='')

In [ ]:
with open('../random_archive/angles_N2.json', 'r', encoding='utf-8') as f:
    loaded_dict = json.load(f)
params = loaded_dict[0][0]
Jz = params['Jz']; Jx = params['Jx']
θ1 = params['θ1']; θ2 = params['θ2']
θ3 = params['θ3']; θ4 = params['θ4']

ale = fn.gate_xyz_disordered(θ1, θ2, Jx/2, -Jx/2, Jz/2, θ3, θ4)
fn.print_matrix(ale)

gate = np.kron(sz, sx)/4
print(f'\n','~'*30,'\nbefore evolution:\n', sep='')
fn.print_matrix(gate, 2)
print_paulis(gate)
gave_evo = ale.conj().T @ gate @ ale
print(f'\n','~'*30,'\nafter evolution:\n', sep='')
fn.print_matrix(gave_evo, 2)
print_paulis(gave_evo)